In [21]:
import os
import email
import base64
import numpy as np
import sklearn 

from bs4 import BeautifulSoup as soup

In [22]:
def extract(file_content):
    #d=file_content.split("Subject:")
    #print d[1]
    b = email.message_from_string(file_content)
    if b.is_multipart():
        a=''
        for payload in b.get_payload():
        # if payload.is_multipart(): ...
             a.join(str(payload.get_payload()))
             return a
    else:
        return b.get_payload()
   



In [23]:
def vectorize(body):
    body=body.lower()
    a="hello"
    b="how"
    c="are"
    d="you"
    l=[a,b,c,d]
    a=[0,0,0,0]
    for i,x in enumerate(l):
        if x in body:
            a[i]=1
    return a
    

In [24]:
def create(folder_name):
    
    file_list=os.listdir(folder_name)
    vector=[]
       
    for i in file_list:
        path=os.path.join(folder_name,i)
        f=open(path,'r').read()
        body=extract(f) 
        vector.append(vectorize(body))
    
    return vector
    
    
    
    

In [25]:
spam_vector=np.asarray(create("spam"))
spam_label=np.ones([len(spam_vector),1])

In [26]:
ham_vector=np.asarray(create("easy_ham"))
ham_label=np.zeros([len(ham_vector),1])

type(ham_vector[3])

numpy.ndarray

In [71]:
hard_ham_vector=np.asarray(create("hard_ham"))
hard_ham_label=np.zeros([len(hard_ham_vector),1])

email_data=np.concatenate((spam_vector,ham_vector),axis=0)
#email_data=np.concatenate((spam_vector,ham_vector,hard_ham_vector),axis=0)

email_label=np.concatenate((spam_label,ham_label),axis=0)



In [89]:
from sklearn.model_selection import train_test_split

train_set,test_set=train_test_split(email_data,test_size=0.1,random_state=42)
train_label,test_label=train_test_split(email_label,test_size=0.1,random_state=42)

In [90]:

from sklearn.metrics import accuracy_score
accuracy_score(test_label,clf.predict(test_set))

0.79411764705882348

In [106]:
def train(clf):
    clf.fit(train_set,train_label.ravel())
    return accuracy_score(test_label,clf.predict(test_set))
    

In [107]:
from sklearn import tree
clf=tree.DecisionTreeClassifier()
clf.fit(train_set,train_label)

print(train(clf))

0.794117647059


In [108]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
print(train(knn))

0.794117647059


In [109]:
from sklearn.naive_bayes import GaussianNB as GNB
nbc=GNB()
print(train(nbc))

0.794117647059


In [110]:
from sklearn import svm
svc=svm.SVC()
print(train(svc))


0.794117647059
